# Logistic Regression

In [57]:
import pandas as pd
import numpy as np

## Taking the AAPL for example to investigate the effect

In [58]:
freq = 'Daily'
stock = 'AAPL'
price = pd.read_csv('../encode_price/'+freq+'/'+stock+'.csv')
y = price.direction.shift(-1).values[:-1]
predictors = pd.read_csv('../predictors/Merged/'+freq+'/'+stock+'.csv',index_col='Date')
predictors.fillna(0,inplace=True)
X = predictors.values[:-1,]

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [74]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)
sm = SMOTE(sampling_strategy='minority',random_state=42,k_neighbors=5)
X_res,y_res = sm.fit_resample(X_train,y_train)

## SMOTE Balanced Dataset

In [75]:
pipe = make_pipeline(StandardScaler(),LogisticRegression(penalty = 'l1',solver='liblinear'))
pipe.fit(X_res,y_res)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.46      0.99      0.63       288
         1.0       0.73      0.03      0.06       346

    accuracy                           0.47       634
   macro avg       0.60      0.51      0.34       634
weighted avg       0.61      0.47      0.32       634



## Original Dataset

In [78]:
pipe = make_pipeline(StandardScaler(),LogisticRegression(penalty = 'l1',solver='liblinear'))
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.50      0.19      0.27       288
         1.0       0.56      0.84      0.67       346

    accuracy                           0.55       634
   macro avg       0.53      0.52      0.47       634
weighted avg       0.53      0.55      0.49       634

